In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Iniciamos con la lectura de los datos

In [2]:
df = pd.read_excel("real_estate_data.xlsx", index_col = [0])

#### Formateo de los nombres de las columnas para remover espacios

In [3]:
def format_cols_spaces(columns):
  return {key:key.replace(" ","_") for key in columns}

In [4]:
columns_replace = format_cols_spaces(df.columns)
df = df.rename(columns = columns_replace)
df_train = df[0:315]
df_test = df[315:]

### Creamos la clase NeuralNetwork

In [16]:
class NeuralNetwork:
  def __init__(self):

    #Inicializacion de pesos
    # set seed
    np.random.seed(42)
    self.W1 = np.random.random((5,6))
    self.b1 = np.random.random((5,1))

    self.W2 = np.random.random((1,5))
    self.b2 = np.random.random((1,1))

  def relu(self,X):
    return np.maximum(0,X)

  def sigmoid(self,X):
    return 1/(1+np.exp(-X))

  def forward(self, X):
    zi = self.W1@X + self.b1
    zi = self.sigmoid(zi)
    y_hat = self.W2@zi + self.b2
    return y_hat.T # y_hat.T es un vector columna
  
  def funcion_objetivo(self, X, y):
    return 0.5*np.mean((self.forward(X) - y)**2)
  
  def numerical_gradient(self, X, y):
      # Calcular el gradiente numérico
      eps = 1e-4
      grads = {}
      for param_name in ["W1", "b1", "W2", "b2"]:
          param = getattr(self, param_name)
          loss_mat = np.zeros_like(param)
          ## For loop para W1
          for i in range(param.shape[0]):
            for j in range(param.shape[1]):
                original_value = param[i, j]
                param[i, j] = original_value + eps
                grad_plus = self.funcion_objetivo(X, y)
                param[i, j] = original_value - eps
                grad_minus = self.funcion_objetivo(X, y)
                param[i, j] = original_value
                loss_mat[i, j] = (grad_plus - grad_minus) / (2 * eps)
          grads[param_name] = loss_mat
          return grads
  
#funcion fit y loop de entrenamiento
  def fit(self, X, y, learning_rate=0.01, epochs=1000):
    # Inicializar historial de pérdidas
    loss_history = []

    # Ciclo de entrenamiento
    for epoch in range(epochs):
        # Calculamos los gradientes
        grads = self.numerical_gradient(X, y)

        # Actualizamos los pesos y bias
        for param_name in ["W1", "b1", "W2", "b2"]:
            param = getattr(self, param_name)
            grad = grads[param_name]
            # param -= learning_rate * grad
            # setattr(self, param_name, param)
            it = np.nditer(param, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index
                param[ix] -= learning_rate * grad[ix]
                it.iternext()

        # Guardamos el valor de la función objetivo para llevar registro de la evolución del entrenamiento
        loss = self.funcion_objetivo(X, y)
        loss_history.append(loss)

        # Opcionalmente, podríamos imprimir una actualización del entrenamiento cada cierto número de epochs
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss}')

    return loss_history



In [17]:
neural_1 = NeuralNetwork()

In [8]:
df_train_X = df_train.drop(['Y_house_price_of_unit_area'], axis = 1)
df_train_y = df_train[['Y_house_price_of_unit_area']]

In [9]:
train_X_values = df_train_X.values.T
train_y_values = df_train_y.values


In [339]:
train_x_shorter = train_X_values[:,0:100]
train_y_shorter = df_train_y.values[0:100]

In [18]:
neural_1.numerical_gradient(train_X_values, train_y_values)

0.3745401188473625
[[0.37464012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452]
 [0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985]
 [0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643]
 [0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184]
 [0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041]]
[[0.37444012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452]
 [0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985]
 [0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643]
 [0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184]
 [0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041]]
0.9507143064099162
[[0.37454012 0.95081431 0.73199394 0.59865848 0.15601864 0.15599452]
 [0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985]
 [0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643]
 [0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.3663

{'W1': array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])}

In [341]:
neural_1.fit(train_X_values, train_y_values)

Epoch 0, Loss: 658.676904679917
Epoch 100, Loss: 97.11958278142293
Epoch 200, Loss: 97.1172115998335
Epoch 300, Loss: 97.11721158982115
Epoch 400, Loss: 97.11721158982111
Epoch 500, Loss: 97.11721158982111
Epoch 600, Loss: 97.11721158982111
Epoch 700, Loss: 97.11721158982111
Epoch 800, Loss: 97.11721158982111
Epoch 900, Loss: 97.11721158982111


[732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 732.653306077788,
 720.0061377975702,
 707.486076736248,
 695.0931228938217,
 682.827276270291,
 670.6885368656561,
 658.676904679917,
 658.676904679917,
 658.676904679917,
 658.676904679917,
 658.676904679917,
 658.676904679917,
 658.676904679917,
 658.676904679917,
 658.676904679917,
 658.676904679917,
 658.676904679917,
 658.676904679917,
 658.6769

In [ ]:
def predict(x):
        y =
        return y